<a href="https://colab.research.google.com/github/SurajitBiswasPhysics/surajit-biswas.github.io/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Step 3: Encode categorical variables
label_encoder = LabelEncoder()
dataset['sex'] = label_encoder.fit_transform(dataset['sex'])  # Encode 'sex'
dataset['smoker'] = label_encoder.fit_transform(dataset['smoker'])  # Encode 'smoker'
data = pd.get_dummies(dataset, columns=['region'], drop_first=True)  # One-hot encode 'region'

In [ ]:
# Step 4: Split the data into features and labels
X = data.drop(columns=['expenses'])  # Features
y = data['expenses']  # Labels

In [ ]:
# Step 5: Split the data into training and testing sets (80% train, 20% test)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert datasets to NumPy arrays for TensorFlow
# Ensure all data is in float32 format
train_dataset = train_dataset.astype('float32')
test_dataset = test_dataset.astype('float32')
train_labels = train_labels.astype('float32')
test_labels = test_labels.astype('float32')

In [ ]:
# Step 6: Define the Keras regression model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(train_dataset.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)  # Output layer
])

In [ ]:
# Step 7: Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.001)  # Default is 0.001; reduce further if necessary
model.compile(optimizer=optimizer, loss='mse', metrics=['mae', 'mse'])

In [ ]:
# Step 8: Train the model
history = model.fit(
    train_dataset, train_labels,
    epochs=5,  # Number of training iterations
    validation_split=0.2,  # Use 20% of the training data for validation
    verbose=1  # Print progress during training
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
